In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler
from IPython.display import display, Markdown
import tiktoken
import random
import data_utils

INFO:datasets:PyTorch version 2.3.0+cu118 available.
PyTorch version 2.3.0+cu118 available.


/opt/conda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
# generator with openai models
Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [23]:
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-4o-mini").encode,
    verbose=True
)
Settings.callback_manager = CallbackManager([token_counter])

## Load data

In [6]:
documents = data_utils.load_md_documents(
    docs_dir='./data/llama-blogs-md',
    docs_metadata='./data/llama_blogs_metadata.json'
)

Num documents: 166


Parsing documents: 100%|██████████| 166/166 [00:00<00:00, 10998.24it/s]


In [24]:
# use random doc only
test_docs = random.sample(documents, 10)

In [25]:
display(Markdown(f"# {test_docs[0].metadata['title']}"))
display(Markdown(test_docs[0].text[:1000]))

# Dynamic Retrieval with LlamaCloud

Building a robust question-answering assistant requires dynamically retrieving
the most relevant information for each query. Sometimes a short snippet
provides a complete answer, while other questions need the full context of an
entire document (e.g. “Give me a summary of this book”).

For questions that fall into the latter category, top-k vector search falls
short; rather than just retrieving the top-k chunks that match, you need to
intentionally retrieve the entire document text and feed that into the context
window of the LLM.

To address this, we’re excited to release **file-level retrieval** for
LlamaCloud, which is a separate retrieval API from our existing **chunk-level
retrieval** capabilities. This retrieval interface then paves the way for us
to build an agent that can dynamically route the question to the right
retrieval interface depending on the properties of the question, which allows
it to more robustly handle different user questions.

##  File-Level Retrieval

This me

In [26]:
test_docs[0].metadata['url']

'https://www.llamaindex.ai/blog/dynamic-retrieval-with-llamacloud'

## Generate questions

In [28]:
generator = TestsetGenerator.from_llama_index(
    generator_llm=Settings.llm,
    critic_llm=Settings.llm,
    embeddings=Settings.embed_model,
)

In [29]:
testset = generator.generate_with_llamaindex_docs(
    test_docs,
    test_size=10,
    distributions={simple: 0.4, reasoning: 0.3, multi_context: 0.3},
    with_debugging_logs=True
)

embedding nodes:   0%|          | 0/56 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 595


embedding nodes:   2%|▏         | 1/56 [00:00<00:33,  1.66it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 905
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 828
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 797
Embedding Token Usage: 868
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 830


embedding nodes:  11%|█         | 6/56 [00:00<00:04, 10.75it/s]

...

[ragas.testset.evolutions.DEBUG] answer generated: {'answer': 'Chunk-level and file-level retrieval offer developers the ability to create more nuanced and accurate systems that adapt to the specific needs of each query.', 'verdict': 1}
[ragas.testset.evolutions.DEBUG] answer generated: {'answer': 'Chunk-level and file-level retrieval offer developers the ability to create more nuanced and accurate systems that adapt to the specific needs of each query.', 'verdict': 1}
Generating:  90%|█████████ | 9/10 [00:34<00:06,  6.39s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ragas.testset.filters.DEBUG] evolution filter: {'reason': 'The first question specifically asks about the outcome of a stock split, while the second question inquires about a broader corporate move and its impact on financial reporting. This indicates different constraints and depths of inquiry.', 'verdict': 0}
[ragas.testset.filters.DEBUG] evolution filter: {'reason': 'The first question specifically asks about the outcome of a stock split, while the second question inquires about a broader corporate move and its impact on financial reporting. This indicates different constraints and depths of inquiry.', 'verdict': 0}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ragas.testset.evolutions.DEBUG] answer generated: {'answer': 'The answer to given question is not present in context', 'verdict': -1}
[ragas.testset.evolutions.DEBUG] answer generated: {'answer': 'The answer to given question is not present in context', 'verdict': -1}
Generating: 100%|██████████| 10/10 [00:39<00:00,  3.93s/it]


In [30]:
df_test = testset.to_pandas()

In [31]:
df_test.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the significance of long context model...,[de4c96e25.pdf' -O 'data/amazon_2022.pdf'\n ...,The significance of long context models in ent...,simple,[{'filename': 'jamba-instruct-s-256k-context-w...,True
1,What is the purpose of structured extraction i...,[Structured extraction from unstructured data ...,The purpose of structured extraction in the co...,simple,[{'filename': 'introducing-llamaextract-beta-s...,True
2,What new feature has been introduced in LlamaC...,"[Greetings, Llama Lovers!\n\nWelcome to this w...",A new feature in LlamaCloud has been introduce...,simple,[{'filename': 'llamaindex-newsletter-2024-08-0...,True
3,What is the process for initiating a fine-tuni...,"[_dataset,\n base_embed_model,\n ...",The process for initiating a fine-tuning job i...,simple,[{'filename': 'fine-tuning-a-linear-adapter-fo...,True
4,What do chunk and file retrieval offer for que...,"[ chunk-level and file-level\nretrieval, devel...",Chunk-level and file-level retrieval offer dev...,reasoning,[{'filename': 'dynamic-retrieval-with-llamaclo...,True


In [16]:
len(df_test)

10

In [32]:
# df_test.to_csv("./data/ragas_testset_03.csv", index=False)

## Token counting

In [37]:
print(
    "Embedding Tokens: ",
    token_counter.total_embedding_token_count,
    "\n",
    "LLM Prompt Tokens: ",
    token_counter.prompt_llm_token_count,
    "\n",
    "LLM Completion Tokens: ",
    token_counter.completion_llm_token_count,
    "\n",
    "Total LLM Token Count: ",
    token_counter.total_llm_token_count,
    "\n",
)

Embedding Tokens:  17890 
 LLM Prompt Tokens:  25702 
 LLM Completion Tokens:  946 
 Total LLM Token Count:  26648 

